In [1]:
import obspy
from obspy.clients.fdsn.mass_downloader import CircularDomain, RectangularDomain, Restrictions, MassDownloader
from obspy.taup import TauPyModel
import pandas as pd


In [2]:
######################################
# Event info
origin_time = obspy.UTCDateTime(2016, 1, 3, 23, 5, 22)
event_lat=24.80360
event_long=93.65050
event_depth=55.0 # km

In [3]:
#######################
## This in case we make a list of stations array wise or in total such then we can ask specifically for data
#data = pd.read_csv('./EU_wilber-stations.txt', sep='|')
#data['Net'].unique()
Array='AU_test'

In [4]:
#####################################
# A typical data download in obspy results in a stations info in xml files that have all the station info including 
# response files etc. The actual waveform data is downloaded separately. This mean when we want to work with waveform 
# data we have to read the station files.
# Drive location to save wavefrom data
mseed_storage = "./data/test/bp_waveforms_"+str(Array)
# Drive location to save station inventory
stationxml_storage = "./data/test/bp_stations_"+str(Array)

In [5]:
###########################
# This stuff defines restrictions on the data for time
start_time_waveform_data=origin_time
end_time_waveform_data=origin_time + 60*60# 3600 seconds


#model = TauPyModel(model="ak135")
#min_travel_time = model.get_travel_times(source_depth_in_km=event_depth,
#                                          distance_in_degree=20,
#                                          phase_list=["P"])
#print(min_travel_time)
#arr = min_travel_time[0]
#starttime = origin_time #+ arr.time - 60.0  # 1 minute before the event

#max_travel_time = model.get_travel_times(source_depth_in_km=event_depth,
#                                          distance_in_degree=95,
#                                          phase_list=["P"])
#print(max_travel_time)

#endtime = origin_time + 800 + (10.0*60.0)  # 10 minutes after the event
#endtime = origin_time + 800 + (10.0*60.0)  # 10 minutes after the event


In [6]:
##########################################################
# This is location from where you want to get the data
# IN principal 
# Circular domain around the epicenter. This will download all data between
# 70 and 90 degrees distance from the epicenter. This module also offers
# rectangular and global domains. More complex domains can be defined by
# inheriting from the Domain class.
#domain = CircularDomain(latitude=event_lat, longitude=event_long,minradius=30.0, maxradius=90.0)
#### For AU
domain = RectangularDomain(minlatitude=-45, maxlatitude=0,
                           minlongitude=100, maxlongitude=170)

In [7]:
restrictions = Restrictions(
    # Get data from origin time of the event to one hour after the
    # event. This defines the temporal bounds of the waveform data.
    starttime=origin_time,
    endtime=origin_time + 60*60,
    # You might not want to deal with gaps in the data. If this setting is
    # True, any trace with a gap/overlap will be discarded.
    reject_channels_with_gaps=True,
    # And you might only want waveforms that have data for at least 95 % of
    # the requested time span. Any trace that is shorter than 95 % of the
    # desired total duration will be discarded.
    minimum_length=1.00,
    ## Network
    # For Australia 
    #network="2P,3M,4N,6C,7G,8J,AU,DU,S1,YS",
    # For Africa
    #network="AF,G,IU,GE,G,II,KV,ZT,NR,GT,8A",
    # For Japan
    #network="CB,IU,KS,KG,JP,G,II",
    # FOr EU
    #network="KZ,II,MD,IU,UD,IM,RO,GE,HE,EE,FN,PL,NO,1G,SJ,UP,MN,HU,CZ,OE,Z3,CR,HF,SL,GR,DK,SX,TH,BW,NI,OX,IV,SI,CH,GU,NR,RN,NL,FR,G,RD,BE,GB,UK,BN,CA,EI,VI,WM,2M",
    # No two stations should be closer than 10 km to each other. This is
    # useful to for example filter out stations that are part of different
    # networks but at the same physical station. Settings this option to
    # zero or None will disable that filtering.
    minimum_interstation_distance_in_m=0.0, #10E3,
    # Only HH or BH channels. If a station has HH channels, those will be
    # downloaded, otherwise the BH. Nothing will be downloaded if it has
    # neither. You can add more/less patterns if you like.
    channel_priorities=["HH[Z]", "BH[Z]", "LH[Z]", "SH[Z]"],
    # Location codes are arbitrary and there is no rule as to which
    # location is best. Same logic as for the previous setting.
    location_priorities=["", "00", "10"])

In [8]:
# No specified providers will result in all known ones being queried.
mdl = MassDownloader()

[2022-03-01 09:47:15,926] - obspy.clients.fdsn.mass_downloader - INFO: Initializing FDSN client(s) for BGR, EMSC, ETH, GEONET, GFZ, ICGC, INGV, IPGP, ISC, KNMI, KOERI, LMU, NCEDC, NIEP, NOA, RESIF, SCEDC, TEXNET, UIB-NORSAR, USGS, USP, ORFEUS, IRIS.
[2022-03-01 09:47:16,527] - obspy.clients.fdsn.mass_downloader - INFO: Cannot use client 'ISC' as it does not have 'dataselect' and/or 'station' services.
[2022-03-01 09:47:17,629] - obspy.clients.fdsn.mass_downloader - INFO: Cannot use client 'EMSC' as it does not have 'dataselect' and/or 'station' services.
[2022-03-01 09:47:17,992] - obspy.clients.fdsn.mass_downloader - INFO: Cannot use client 'USGS' as it does not have 'dataselect' and/or 'station' services.
[2022-03-01 09:47:18,547] - obspy.clients.fdsn.mass_downloader - INFO: Successfully initialized 20 client(s): BGR, ETH, GEONET, GFZ, ICGC, INGV, IPGP, KNMI, KOERI, LMU, NCEDC, NIEP, NOA, RESIF, SCEDC, TEXNET, UIB-NORSAR, USP, ORFEUS, IRIS.


In [9]:
mdl.download(domain, restrictions,threads_per_client=3, mseed_storage=mseed_storage,
             stationxml_storage=stationxml_storage)


[2022-03-01 09:47:18,562] - obspy.clients.fdsn.mass_downloader - INFO: Total acquired or preexisting stations: 0
[2022-03-01 09:47:18,562] - obspy.clients.fdsn.mass_downloader - INFO: Client 'BGR' - Requesting unreliable availability.
[2022-03-01 09:47:18,578] - obspy.clients.fdsn.mass_downloader - INFO: Client 'BGR' - No data available for request.
[2022-03-01 09:47:18,578] - obspy.clients.fdsn.mass_downloader - INFO: Client 'BGR' - No data available.
[2022-03-01 09:47:18,578] - obspy.clients.fdsn.mass_downloader - INFO: Total acquired or preexisting stations: 0
[2022-03-01 09:47:18,578] - obspy.clients.fdsn.mass_downloader - INFO: Client 'ETH' - Requesting unreliable availability.
[2022-03-01 09:47:18,678] - obspy.clients.fdsn.mass_downloader - INFO: Client 'ETH' - No data available for request.
[2022-03-01 09:47:18,678] - obspy.clients.fdsn.mass_downloader - INFO: Client 'ETH' - No data available.
[2022-03-01 09:47:18,678] - obspy.clients.fdsn.mass_downloader - INFO: Total acquired 

[2022-03-01 09:47:24,735] - obspy.clients.fdsn.mass_downloader - INFO: Client 'IPGP' - Successfully downloaded './data/test/bp_stations_AU_test\G.DZM.xml'.
[2022-03-01 09:47:25,020] - obspy.clients.fdsn.mass_downloader - INFO: Client 'IPGP' - Successfully downloaded './data/test/bp_stations_AU_test\G.SANVU.xml'.
[2022-03-01 09:47:25,020] - obspy.clients.fdsn.mass_downloader - INFO: Client 'IPGP' - Downloaded 4 station files [0.0 MB] in 1.3 seconds [32.50 KB/sec].
[2022-03-01 09:47:25,020] - obspy.clients.fdsn.mass_downloader - INFO: Total acquired or preexisting stations: 9
[2022-03-01 09:47:25,020] - obspy.clients.fdsn.mass_downloader - INFO: Client 'KNMI' - Requesting unreliable availability.
[2022-03-01 09:47:25,067] - obspy.clients.fdsn.mass_downloader - INFO: Client 'KNMI' - No data available for request.
[2022-03-01 09:47:25,067] - obspy.clients.fdsn.mass_downloader - INFO: Client 'KNMI' - No data available.
[2022-03-01 09:47:25,067] - obspy.clients.fdsn.mass_downloader - INFO: T

[2022-03-01 09:47:32,214] - obspy.clients.fdsn.mass_downloader - INFO: Client 'USP' - Status for 2 time intervals/channels before downloading: NEEDS_DOWNLOADING
[2022-03-01 09:47:32,634] - obspy.clients.fdsn.mass_downloader - INFO: Client 'USP' - No data available for request.
[2022-03-01 09:47:32,634] - obspy.clients.fdsn.mass_downloader - INFO: Client 'USP' - Launching basic QC checks...
[2022-03-01 09:47:32,650] - obspy.clients.fdsn.mass_downloader - INFO: Client 'USP' - Downloaded 0.0 MB [0.00 KB/sec] of data, 0.0 MB of which were discarded afterwards.
[2022-03-01 09:47:32,650] - obspy.clients.fdsn.mass_downloader - INFO: Client 'USP' - Status for 2 time intervals/channels after downloading: DOWNLOAD_FAILED
[2022-03-01 09:47:32,650] - obspy.clients.fdsn.mass_downloader - INFO: Client 'USP' - No station information to download.
[2022-03-01 09:47:32,650] - obspy.clients.fdsn.mass_downloader - INFO: Client 'USP' - No data could be downloaded.
[2022-03-01 09:47:32,650] - obspy.clients.

[2022-03-01 09:50:44,784] - obspy.clients.fdsn.mass_downloader - INFO: File './data/test/bp_waveforms_AU_test\AU.MULG..BHZ__20160103T230522Z__20160104T000522Z.mseed' has only 3599.97 seconds of data. 3600.00 are required. File will be deleted.
[2022-03-01 09:50:44,787] - obspy.clients.fdsn.mass_downloader - INFO: File './data/test/bp_waveforms_AU_test\AU.MUN..BHZ__20160103T230522Z__20160104T000522Z.mseed' has only 3599.97 seconds of data. 3600.00 are required. File will be deleted.
[2022-03-01 09:50:44,791] - obspy.clients.fdsn.mass_downloader - INFO: File './data/test/bp_waveforms_AU_test\AU.NFK..BHZ__20160103T230522Z__20160104T000522Z.mseed' has only 3599.97 seconds of data. 3600.00 are required. File will be deleted.
[2022-03-01 09:50:44,794] - obspy.clients.fdsn.mass_downloader - INFO: File './data/test/bp_waveforms_AU_test\AU.OOD..BHZ__20160103T230522Z__20160104T000522Z.mseed' has only 3599.97 seconds of data. 3600.00 are required. File will be deleted.
[2022-03-01 09:50:44,798] -

[2022-03-01 09:50:44,964] - obspy.clients.fdsn.mass_downloader - INFO: File './data/test/bp_waveforms_AU_test\S1.AUALB..HHZ__20160103T230522Z__20160104T000522Z.mseed' has 2 traces and thus contains gaps or overlaps. Will be deleted.
[2022-03-01 09:50:44,990] - obspy.clients.fdsn.mass_downloader - INFO: File './data/test/bp_waveforms_AU_test\S1.AUCSH..HHZ__20160103T230522Z__20160104T000522Z.mseed' has 5 traces and thus contains gaps or overlaps. Will be deleted.
[2022-03-01 09:50:44,996] - obspy.clients.fdsn.mass_downloader - INFO: File './data/test/bp_waveforms_AU_test\S1.AUDCS..HHZ__20160103T230522Z__20160104T000522Z.mseed' has 3 traces and thus contains gaps or overlaps. Will be deleted.
[2022-03-01 09:50:45,007] - obspy.clients.fdsn.mass_downloader - INFO: File './data/test/bp_waveforms_AU_test\S1.AUHPC..HHZ__20160103T230522Z__20160104T000522Z.mseed' has only 3567.99 seconds of data. 3600.00 are required. File will be deleted.
[2022-03-01 09:50:45,014] - obspy.clients.fdsn.mass_down

[2022-03-01 09:50:48,382] - obspy.clients.fdsn.mass_downloader - INFO: Client 'IRIS' - Successfully downloaded './data/test/bp_stations_AU_test\AU.WR1.xml'.
[2022-03-01 09:50:48,399] - obspy.clients.fdsn.mass_downloader - INFO: Client 'IRIS' - Successfully downloaded './data/test/bp_stations_AU_test\AU.WR8.xml'.
[2022-03-01 09:50:48,404] - obspy.clients.fdsn.mass_downloader - INFO: Client 'IRIS' - Successfully downloaded './data/test/bp_stations_AU_test\S1.AUKAT.xml'.
[2022-03-01 09:50:48,730] - obspy.clients.fdsn.mass_downloader - INFO: Client 'IRIS' - Successfully downloaded './data/test/bp_stations_AU_test\AU.WR2.xml'.
[2022-03-01 09:50:48,766] - obspy.clients.fdsn.mass_downloader - INFO: Client 'IRIS' - Successfully downloaded './data/test/bp_stations_AU_test\S1.AUKSC.xml'.
[2022-03-01 09:50:48,769] - obspy.clients.fdsn.mass_downloader - INFO: Client 'IRIS' - Successfully downloaded './data/test/bp_stations_AU_test\AU.WR9.xml'.
[2022-03-01 09:50:49,077] - obspy.clients.fdsn.mass_do

[2022-03-01 09:50:55,115] - obspy.clients.fdsn.mass_downloader - INFO: 0 StationXML files [0.0 MB] already existed.
[2022-03-01 09:50:55,116] - obspy.clients.fdsn.mass_downloader - INFO: Client 'BGR' - Acquired 0 MiniSEED files [0.0 MB].
[2022-03-01 09:50:55,116] - obspy.clients.fdsn.mass_downloader - INFO: Client 'BGR' - Acquired 0 StationXML files [0.0 MB].
[2022-03-01 09:50:55,116] - obspy.clients.fdsn.mass_downloader - INFO: Client 'ETH' - Acquired 0 MiniSEED files [0.0 MB].
[2022-03-01 09:50:55,117] - obspy.clients.fdsn.mass_downloader - INFO: Client 'ETH' - Acquired 0 StationXML files [0.0 MB].
[2022-03-01 09:50:55,118] - obspy.clients.fdsn.mass_downloader - INFO: Client 'GEONET' - Acquired 3 MiniSEED files [1.2 MB].
[2022-03-01 09:50:55,118] - obspy.clients.fdsn.mass_downloader - INFO: Client 'GEONET' - Acquired 3 StationXML files [0.0 MB].
[2022-03-01 09:50:55,119] - obspy.clients.fdsn.mass_downloader - INFO: Client 'GFZ' - Acquired 2 MiniSEED files [0.9 MB].
[2022-03-01 09:50:

{'BGR': <obspy.clients.fdsn.mass_downloader.download_helpers.ClientDownloadHelper at 0x222d14696c8>,
 'ETH': <obspy.clients.fdsn.mass_downloader.download_helpers.ClientDownloadHelper at 0x222d139a8c8>,
 'GEONET': <obspy.clients.fdsn.mass_downloader.download_helpers.ClientDownloadHelper at 0x222d12dc2c8>,
 'GFZ': <obspy.clients.fdsn.mass_downloader.download_helpers.ClientDownloadHelper at 0x222d25973c8>,
 'ICGC': <obspy.clients.fdsn.mass_downloader.download_helpers.ClientDownloadHelper at 0x222d2562848>,
 'INGV': <obspy.clients.fdsn.mass_downloader.download_helpers.ClientDownloadHelper at 0x222d1468308>,
 'IPGP': <obspy.clients.fdsn.mass_downloader.download_helpers.ClientDownloadHelper at 0x222d37a5148>,
 'KNMI': <obspy.clients.fdsn.mass_downloader.download_helpers.ClientDownloadHelper at 0x222d399bd48>,
 'KOERI': <obspy.clients.fdsn.mass_downloader.download_helpers.ClientDownloadHelper at 0x222d399bd88>,
 'LMU': <obspy.clients.fdsn.mass_downloader.download_helpers.ClientDownloadHelper 